In [2]:
import pandas as pd
import numpy as np

In [3]:
games = pd.read_csv('results_afl_tables_12TO23.csv')
# games
ladder = pd.read_csv('LADDER_15TO23.csv')
# ladder 
last_ladder_position = pd.read_excel('last_ladder_position.xlsx')
# last_ladder_position 
pis = pd.read_csv('PIS_15TO23.csv')

C:\Users\carlo\miniconda3\envs\ie-ml\lib\site-packages\IPython\core\interactiveshell.py:3441: DtypeWarning: Columns (15,49,89) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


# PROCESSING_Games

In [4]:
#rename
results_col_names = {
    'Game':'GAME',
    'Date':'DATE',
    'Round':'ROUND',
    'Home.Team':'HOME_TEAM',
    'Home.Goals':'HOME_GOALS',
    'Home.Behinds':'HOME_BEHINDS',
    'Home.Points':'HOME_POINTS',
    'Away.Team':'AWAY_TEAM',
    'Away.Goals':'AWAY_GOALS',
    'Away.Behinds':'AWAY_BEHINDS',
    'Away.Points':'AWAY_POINTS',
    'Venue':'VENUE',
    'Margin':'MARGIN',
    'Season':'SEASON',
    'Round.Type':'ROUND_TYPE',
    'Round.Number':'ROUND_NUMBER'
}

games = games.rename(columns=results_col_names)
games = games[games['SEASON']>2014]
print(games.shape)
games.SEASON.value_counts()
# games.dtypes

(1826, 16)


2023    216
2016    207
2017    207
2018    207
2019    207
2021    207
2022    207
2015    206
2020    162
Name: SEASON, dtype: int64

In [5]:
team_names_games = {'Richmond':'RICH',
                    'Collingwood':'COLL',
                    'Melbourne':'MELB',
                    'Adelaide':'ADEL',
                    'Adelaide Crows':'ADEL',
                    'Essendon':'ESS',
                    'Brisbane Lions':'BL',
                    'North Melbourne':'NMFC',
                    'GWS':'GWS',
                    'GWS Giants':'GWS',
                    'West Coast':'WCE',
                    'West Coast Eagles':'WCE',
                    'Carlton':'CARL',
                    'Geelong':'GEEL',
                    'Geelong Cats':'GEEL',
                    'Sydney':'SYD',
                    'Sydney Swans':'SYD',
                    'Port Adelaide':'PORT',
                    'St Kilda':'STK',
                    'Gold Coast':'GCFC',
                    'Gold Coast Suns':'GCFC',
                    'Hawthorn':'HAW',
                    'Footscray':'WB',
                    'Western Bulldogs':'WB',
                    'Fremantle':'FRE'}

In [6]:
def rename_teams(data):
    
    data['HOME_TEAM'] = data['HOME_TEAM'].map(team_names_games)   
    data['AWAY_TEAM'] = data['AWAY_TEAM'].map(team_names_games)
    
    return data

def wins_home_away(data):
    
    data['HOME_WIN'] = data['MARGIN'].apply(lambda x: 1 if x > 0 else 0)
    data['AWAY_WIN'] = data['MARGIN'].apply(lambda x: 1 if x < 0 else 0)
    data['DRAW'] = data['MARGIN'].apply(lambda x: 1 if x == 0 else 0)
    
    return data

def ladders_ly(games, last_ladder_position):
    
    """
    For each team, identifies its ladder position last season.
    
    Parameters:
        - games: DataFrame with the game results
        - last_ladder_position: DataFrame with the ladder position each season 
        for every team.
    
    Returns:
        - games: DataFrame with the game results with updated ladder positions 
        in the previous season.
    
    """
    
    games['HOME_LADDERLY_POSITION'] = None
    games['AWAY_LADDERLY_POSITION'] = None
    
    
    for index, row in games.iterrows():
        
        season = row['SEASON']
        home_team = row['HOME_TEAM']
        away_team = row['AWAY_TEAM']
        
        home_corresponding_position = last_ladder_position.loc[
            last_ladder_position['TEAM'] == home_team, f'LADDER_PREPO_{str(season)}'].values[0]

        away_corresponding_position = last_ladder_position.loc[
            last_ladder_position['TEAM'] == away_team, f'LADDER_PREPO_{str(season)}'].values[0]

        
        games.at[index, 'HOME_LADDERLY_POSITION'] = home_corresponding_position
        games.at[index, 'AWAY_LADDERLY_POSITION'] = away_corresponding_position

    
    return games

In [7]:
games = wins_home_away(games)
games = rename_teams(games)
games = ladders_ly(games, last_ladder_position)

In [11]:
games_HOME = games[['HOME_TEAM', 'HOME_WIN', 'SEASON']].rename(columns={'HOME_TEAM': 'TEAM', 'HOME_WIN': 'WIN'})
games_AWAY = games[['AWAY_TEAM', 'AWAY_WIN', 'SEASON']].rename(columns={'AWAY_TEAM': 'TEAM', 'AWAY_WIN': 'WIN'})
df_concatenated = pd.concat([games_HOME, games_AWAY], ignore_index=True)

In [42]:
# Run this cell to check which teams should be taken as reference

df_concatenated[df_concatenated['SEASON'].isin([2023, 2020, 2021, 2022])].\
    groupby(by = ['TEAM'])['WIN'].sum().sort_values(ascending=False)

TEAM
BL      66
GEEL    62
MELB    61
PORT    60
COLL    54
WB      52
SYD     50
RICH    47
STK     45
FRE     43
CARL    42
GWS     41
ESS     35
GCFC    31
ADEL    29
HAW     27
WCE     27
NMFC    12
Name: WIN, dtype: int64

In [ ]:
def prep_st_BT(games):
    
    """
    Prepares datasets for modelling the standard BT expansion.
    
    Parameters:
        - games: DataFrame with the game results
    
    Details:
        - Excludes draws
        - For each season, it groups by HOME-AWAY pairs and takes win totals
        - Creates a dataset with all available results.
        - All datasets are appended to a list.
    
    Returns:
        - dataset_list: list with processed datasets.
    
    """
    dataset_list = []
    
    games_f = games[games['DRAW'] == 0]
    tempos = games_f['SEASON'].unique()
    
    for season in tempos:
        print(season)
        season_df = games_f[games_f['SEASON'] == season]
        print(season_df.shape)
        BT_season = season_df.groupby(['HOME_TEAM', 'AWAY_TEAM']).agg({
        'HOME_WIN': 'sum',
        'AWAY_WIN': 'sum'}).reset_index()
        dataset_list.append(BT_season)
    
    games_full_ST = games_f.groupby(['HOME_TEAM', 'AWAY_TEAM']).agg({
        'HOME_WIN': 'sum',
        'AWAY_WIN': 'sum'}).reset_index()
    
    dataset_list.append(games_full_ST)
    print(len(dataset_list))
    return dataset_list
    

In [ ]:
list_STBT = prep_st_BT(games)

In [ ]:
def save2excel(dataset_list):
    for i, data in enumerate(dataset_list):
        season = 2015 + i
        filename = f'AFL_ST_{season}.xlsx'
        data.to_excel(filename, index=False)
        print(f"Saved data for season {season} to {filename}")

In [ ]:
save2excel(list_STBT)

In [ ]:
def prep_st_BT_test(games):
    
    """
    Prepares datasets for testing the standard BT expansion.
    
    Parameters:
        - games: DataFrame with the game results
    
    Details:
        - Follows the structure in prep_st_BT
    
    Returns:
        - dataset_list: list with processed datasets.
    
    """
    
    cols = ['HOME_TEAM', 'AWAY_TEAM', 'HOME_WIN', 'AWAY_WIN']
    dataset_list = []
    
    games_f = games[games['DRAW'] == 0]
    tempos = games_f['SEASON'].unique()
    
    for season in tempos:
        print(season)
        season_df = games_f[games_f['SEASON'] == season][cols]
        print(season_df.shape)
        dataset_list.append(season_df)
    
    games_full_ST = games_f[cols]
    
    dataset_list.append(games_full_ST)
    print(len(dataset_list))
    return dataset_list

def save2excell(dataset_list):
    for i, data in enumerate(dataset_list):
        season = 2015 + i
        filename = f'AFL_ST_{season}_TEST.xlsx'
        data.to_excel(filename, index=False)
        print(f"Saved data for season {season} to {filename}")

In [ ]:
list_STBT_TEST = prep_st_BT_test(games)
save2excell(list_STBT_TEST)

# PROCESSING_PIs

In [ ]:
pis.head()

In [ ]:
pis.columns

In [ ]:
pis['team.name'].unique()

In [8]:
pis_actions = [
    'round.roundNumber', 'SEASON', 'team.name',
    'disposals', 'extendedStats.effectiveDisposals','disposalEfficiency',
    'kicks', 'extendedStats.effectiveKicks', 'extendedStats.kickEfficiency',
    'handballs',
    'extendedStats.kickToHandballRatio',
    'marks', 'marksInside50', 'contestedMarks', 'extendedStats.marksOnLead','extendedStats.interceptMarks',
    'bounces',
    'metresGained'
]

# offensive: attacking or wins
pis_offensive = [
    'totalPossessions', 'contestedPossessions', 'uncontestedPossessions','extendedStats.contestedPossessionRate',
    'extendedStats.groundBallGets', 'extendedStats.f50GroundBallGets',
    'inside50s',
    'shotsAtGoal','goals', 'goalAccuracy','extendedStats.scoreLaunches',
    'clearances.totalClearances', 'clearances.centreClearances', 'clearances.stoppageClearances',
    'hitouts','extendedStats.hitoutsToAdvantage', 'extendedStats.hitoutWinPercentage', 'extendedStats.hitoutToAdvantageRate',
    'extendedStats.contestOffOneOnOnes', 'extendedStats.contestOffWins', 'extendedStats.contestOffWinsPercentage'
]

#defensive: defending or losses/errors
#turnover - loosing possession to opositiob
pis_defensive = [
    'tackles', 'tacklesInside50',
    'onePercenters',
    'clangers', #error
    'freesAgainst', #error
    'rebound50s',
    'turnovers', #error
    'intercepts',
    'extendedStats.pressureActs', 'extendedStats.defHalfPressureActs',
    'extendedStats.spoils',
    'extendedStats.contestDefOneOnOnes', 'extendedStats.contestDefLosses', 'extendedStats.contestDefLossPercentage'
]

In [9]:
pis_short = pis[pis_actions + pis_offensive + pis_defensive]
pis_short.head()

,round.roundNumber,SEASON,team.name,disposals,extendedStats.effectiveDisposals,disposalEfficiency,kicks,extendedStats.effectiveKicks,extendedStats.kickEfficiency,handballs,...,freesAgainst,rebound50s,turnovers,intercepts,extendedStats.pressureActs,extendedStats.defHalfPressureActs,extendedStats.spoils,extendedStats.contestDefOneOnOnes,extendedStats.contestDefLosses,extendedStats.contestDefLossPercentage
0,1,2015,Carlton,25.0,18.0,72.0,11.0,7.0,63.6,14.0,...,0.0,0.0,4.0,1.0,20.0,4.0,2.0,0.0,0.0,0.0
1,1,2015,Carlton,18.0,16.0,88.9,9.0,7.0,77.8,9.0,...,2.0,0.0,3.0,1.0,16.0,7.0,0.0,0.0,0.0,0.0
2,1,2015,Carlton,14.0,9.0,64.3,4.0,1.0,25.0,10.0,...,0.0,2.0,3.0,1.0,10.0,6.0,0.0,0.0,0.0,0.0
3,1,2015,Carlton,19.0,15.0,78.9,6.0,4.0,66.7,13.0,...,1.0,2.0,3.0,6.0,12.0,6.0,1.0,2.0,1.0,50.0
4,1,2015,Carlton,23.0,21.0,91.3,15.0,14.0,93.3,8.0,...,0.0,5.0,2.0,12.0,17.0,12.0,1.0,1.0,0.0,0.0


In [10]:
pis_col_names = {
    'round.roundNumber':'ROUND_NUMBER',
    'team.name':'TEAM',
    'disposals':'DISPOSALS',
    'extendedStats.effectiveDisposals':'DISPOSALS_EFFECTIVE',
    'disposalEfficiency':'DISPOSALS_EFFICIENCY', #
    'kicks':'KICKS',
    'extendedStats.effectiveKicks':'KICKS_EFFECTIVE',
    'extendedStats.kickEfficiency':'KICKS_EFFICIENCY', #
    'handballs':'HANDBALLS',
    'extendedStats.kickToHandballRatio':'KICK2HANDBALL', #
    'marks':'MARKS',
    'marksInside50':'MARKS_INSIDE50',
    'contestedMarks':'MARKS_CONTESTED',
    'extendedStats.marksOnLead':'MARKS_ONLEAD',
    'extendedStats.interceptMarks':'MARKS_INTERCEPT',
    'bounces':'BOUNCES',
    'metresGained':'METRES_GAINED',
    
    'totalPossessions':'POSESSIONS',
    'contestedPossessions':'POSESSIONS_CONTESTED',
    'uncontestedPossessions':'POSESSIONS_UNCONTESTED',
    'extendedStats.contestedPossessionRate':'POSESSIONS_CONTESTED_RATE', #
    'extendedStats.groundBallGets':'GETS_GROUNDBALL',
    'extendedStats.f50GroundBallGets':'GETS_GROUNDBALL50',
    'inside50s':'INSIDE50',
    'shotsAtGoal':'GOALS_SHOTS',
    'goals':'GOALS',
    'goalAccuracy':'GOALS_ACCURACY', #
    'extendedStats.scoreLaunches':'SCORE_LAUNCHES',
    'clearances.totalClearances':'CLEARANCES',
    'clearances.centreClearances':'CLEARANCES_CENTRE',
    'clearances.stoppageClearances':'CLEARANCES_STOPPAGE',
    'hitouts':'HITOUTS',
    'extendedStats.hitoutsToAdvantage':'HITOUTS_ADVANTAGE',
    'extendedStats.hitoutWinPercentage':'HITOUTS_WIN_RATE',
    'extendedStats.hitoutToAdvantageRate':'HITOUTS_ADVANTAGE_RATE', #
    'extendedStats.contestOffOneOnOnes':'CONTEST_OFFENSIVE',
    'extendedStats.contestOffWins':'CONTEST_OFFENSIVE_WIN',
    'extendedStats.contestOffWinsPercentage':'CONTEST_OFFENSIVE_WIN_RATE',#
    
    'tackles':'TACKLES',
    'tacklesInside50':'TACKLES_INSIDE50',
    'onePercenters':'ONE_PERCENTERS',
    'clangers':'CLANGERS', #error
    'freesAgainst':'FREES_AGAINST', #error
    'rebound50s':'REBOUND_INSIDE50S',
    'turnovers':'TURNOVERS', #error
    'intercepts':'INTERCEPTS',
    'extendedStats.pressureActs':'PRESSURE',
    'extendedStats.defHalfPressureActs':'PRESSURE_DEFENSEHALF',
    'extendedStats.spoils':'SPOILS',
    'extendedStats.contestDefOneOnOnes':'CONTEST_DEFENSIVE',
    'extendedStats.contestDefLosses':'CONTEST_DEFENSIVE_LOSS',
    'extendedStats.contestDefLossPercentage':'CONTEST_DEFENSIVE_LOSS_RATE' # 
}

In [11]:
pis_short = pis_short.rename(columns=pis_col_names)
pis_short['TEAM'] = pis_short['TEAM'].map(team_names_games)  
pis_short.shape

(81605, 53)

In [ ]:
pis_short.columns

In [12]:
pis_summary = pis_short.groupby(['SEASON', 'ROUND_NUMBER', 'TEAM']).agg(
    DISPOSALS = ('DISPOSALS', 'sum'),
    DISPOSALS_EFFECTIVE = ('DISPOSALS_EFFECTIVE', 'sum'),
    KICKS = ('KICKS', 'sum'),
    KICKS_EFFECTIVE = ('KICKS_EFFECTIVE', 'sum'),
    HANDBALLS = ('HANDBALLS', 'sum'),
    MARKS = ('MARKS', 'sum'),
    MARKS_INSIDE50 = ('MARKS_INSIDE50', 'sum'),
    MARKS_CONTESTED = ('MARKS_CONTESTED', 'sum'),
    MARKS_ONLEAD = ('MARKS_ONLEAD', 'sum'),
    MARKS_INTERCEPT = ('MARKS_INTERCEPT', 'sum'),
    BOUNCES = ('BOUNCES', 'sum'),
    METRES_GAINED = ('METRES_GAINED', 'sum'),
    POSESSIONS = ('POSESSIONS', 'sum'),
    POSESSIONS_CONTESTED = ('POSESSIONS_CONTESTED', 'sum'),
    POSESSIONS_UNCONTESTED = ('POSESSIONS_UNCONTESTED', 'sum'),
    GETS_GROUNDBALL = ('GETS_GROUNDBALL', 'sum'),
    GETS_GROUNDBALL50 = ('GETS_GROUNDBALL50', 'sum'),
    INSIDE50 = ('INSIDE50', 'sum'),
    GOALS_SHOTS = ('GOALS_SHOTS', 'sum'),
    GOALS = ('GOALS', 'sum'),
    SCORE_LAUNCHES = ('SCORE_LAUNCHES', 'sum'),
    CLEARANCES = ('CLEARANCES', 'sum'),
    CLEARANCES_CENTRE = ('CLEARANCES_CENTRE', 'sum'),
    CLEARANCES_STOPPAGE = ('CLEARANCES_STOPPAGE', 'sum'),
    HITOUTS = ('HITOUTS', 'sum'),
    HITOUTS_ADVANTAGE = ('HITOUTS_ADVANTAGE', 'sum'),
    HITOUTS_WIN_RATE = ('HITOUTS_WIN_RATE', lambda x: (x * pis_short.loc[x.index, 'HITOUTS'])\
                        .sum() / pis_short.loc[x.index, 'HITOUTS'].sum()),
    CONTEST_OFFENSIVE = ('CONTEST_OFFENSIVE', 'sum'),
    CONTEST_OFFENSIVE_WIN = ('CONTEST_OFFENSIVE_WIN', 'sum'),
    TACKLES = ('TACKLES', 'sum'),
    TACKLES_INSIDE50 = ('TACKLES_INSIDE50', 'sum'),
    ONE_PERCENTERS = ('ONE_PERCENTERS', 'sum'),
    CLANGERS = ('CLANGERS', 'sum'),
    FREES_AGAINST = ('FREES_AGAINST', 'sum'),
    REBOUND_INSIDE50S = ('REBOUND_INSIDE50S', 'sum'),
    TURNOVERS = ('TURNOVERS', 'sum'),
    INTERCEPTS = ('INTERCEPTS', 'sum'),
    PRESSURE = ('PRESSURE', 'sum'),
    PRESSURE_DEFENSEHALF = ('PRESSURE_DEFENSEHALF', 'sum'),
    SPOILS = ('SPOILS', 'sum'),
    CONTEST_DEFENSIVE = ('CONTEST_DEFENSIVE', 'sum'),
    CONTEST_DEFENSIVE_LOSS = ('CONTEST_DEFENSIVE_LOSS', 'sum')
    
).reset_index()


pis_summary.shape

(3652, 45)

In [ ]:
pis_summary

In [13]:
pis_cols_csum = ['DISPOSALS', 'DISPOSALS_EFFECTIVE',
       'KICKS', 'KICKS_EFFECTIVE', 'HANDBALLS', 'MARKS', 'MARKS_INSIDE50',
       'MARKS_CONTESTED', 'MARKS_ONLEAD', 'MARKS_INTERCEPT', 'BOUNCES',
       'METRES_GAINED', 'POSESSIONS', 'POSESSIONS_CONTESTED',
       'POSESSIONS_UNCONTESTED', 'GETS_GROUNDBALL', 'GETS_GROUNDBALL50',
       'INSIDE50', 'GOALS_SHOTS', 'SCORE_LAUNCHES', 'CLEARANCES',
       'CLEARANCES_CENTRE', 'CLEARANCES_STOPPAGE', 
       'HITOUTS_ADVANTAGE', 'HITOUTS_WIN_RATE', 
       'CONTEST_OFFENSIVE_WIN', 'TACKLES', 'TACKLES_INSIDE50',
       'ONE_PERCENTERS', 'CLANGERS', 'FREES_AGAINST', 'REBOUND_INSIDE50S',
       'TURNOVERS', 'INTERCEPTS', 'PRESSURE', 'PRESSURE_DEFENSEHALF', 'SPOILS','CONTEST_DEFENSIVE_LOSS',
        'GOALS', 'HITOUTS', 'CONTEST_OFFENSIVE', 'CONTEST_DEFENSIVE']
pis_cols_avg = ['DISPOSALS_EFFICIENCY',
       'KICKS_EFFICIENCY', 'KICK2HANDBALL', 'POSESSIONS_CONTESTED_RATE',
       'GOALS_ACCURACY', 'HITOUTS_ADVANTAGE_RATE',
       'CONTEST_OFFENSIVE_WIN_RATE', 'CONTEST_DEFENSIVE_LOSS_RATE']

In [14]:
pis_sorted = pis_summary.sort_values(by=['SEASON', 'TEAM', 'ROUND_NUMBER'])
def custom_rolling_L4sum(x):
    """
    Calculates cumulatives of PIs over the last 4 games. If less than 4 games
    have been played, it calculates the cumulative sum until that point.
    
    """
    
    if len(x) >= 4:
        return x.rolling(window=4, min_periods=1).sum() 
    else:
        return x.cumsum()  

for col in pis_cols_csum:
    pis_sorted[col + '_CSUM'] = pis_sorted.groupby(['SEASON', 'TEAM'])[col].cumsum()
    pis_sorted[col + '_L4_CSUM'] = pis_sorted.groupby(['SEASON', 'TEAM'])[col].transform(custom_rolling_L4sum)

pis_sorted = pis_sorted.drop(columns=pis_cols_csum)
pis_sorted.shape

(3652, 87)

In [15]:
pis_sorted['DISPOSALS_EFFICIENCY_CSUM'] = pis_sorted['DISPOSALS_EFFECTIVE_CSUM']/pis_sorted['DISPOSALS_CSUM']*100
pis_sorted['KICKS_EFFICIENCY_CSUM'] = pis_sorted['KICKS_EFFECTIVE_CSUM']/pis_sorted['KICKS_CSUM']*100
pis_sorted['KICK2HANDBALL_CSUM'] = pis_sorted['KICKS_CSUM']/pis_sorted['HANDBALLS_CSUM']
pis_sorted['POSESSIONS_CONTESTED_RATE_CSUM'] = pis_sorted['POSESSIONS_CONTESTED_CSUM']/pis_sorted['POSESSIONS_CSUM']*100
pis_sorted['GOALS_ACCURACY_CSUM'] = pis_sorted['GOALS_CSUM']/pis_sorted['GOALS_SHOTS_CSUM']*100
pis_sorted['HITOUTS_ADVANTAGE_RATE_CSUM'] = pis_sorted['HITOUTS_ADVANTAGE_CSUM']/pis_sorted['HITOUTS_CSUM']*100
pis_sorted['CONTEST_OFFENSIVE_WIN_RATE_CSUM'] = pis_sorted['CONTEST_OFFENSIVE_WIN_CSUM']/pis_sorted['CONTEST_OFFENSIVE_CSUM']*100
pis_sorted['CONTEST_DEFENSIVE_LOSS_RATE_CSUM'] = pis_sorted['CONTEST_DEFENSIVE_LOSS_CSUM']/pis_sorted['CONTEST_DEFENSIVE_CSUM']*100

pis_sorted['CONTEST_OFFENSIVE_WIN_RATE_CSUM'] = pis_sorted['CONTEST_OFFENSIVE_WIN_RATE_CSUM'].fillna(0)
pis_sorted['CONTEST_DEFENSIVE_LOSS_RATE_CSUM'] = pis_sorted['CONTEST_DEFENSIVE_LOSS_RATE_CSUM'].fillna(0)

pis_sorted['DISPOSALS_EFFICIENCY_L4_CSUM'] = pis_sorted['DISPOSALS_EFFECTIVE_L4_CSUM']/pis_sorted['DISPOSALS_L4_CSUM']*100
pis_sorted['KICKS_EFFICIENCY_L4_CSUM'] = pis_sorted['KICKS_EFFECTIVE_L4_CSUM']/pis_sorted['KICKS_L4_CSUM']*100
pis_sorted['KICK2HANDBALL_L4_CSUM'] = pis_sorted['KICKS_L4_CSUM']/pis_sorted['HANDBALLS_L4_CSUM']
pis_sorted['POSESSIONS_CONTESTED_RATE_L4_CSUM'] = pis_sorted['POSESSIONS_CONTESTED_L4_CSUM']/pis_sorted['POSESSIONS_L4_CSUM']*100
pis_sorted['GOALS_ACCURACY_L4_CSUM'] = pis_sorted['GOALS_L4_CSUM']/pis_sorted['GOALS_SHOTS_L4_CSUM']*100
pis_sorted['HITOUTS_ADVANTAGE_RATE_L4_CSUM'] = pis_sorted['HITOUTS_ADVANTAGE_L4_CSUM']/pis_sorted['HITOUTS_L4_CSUM']*100
pis_sorted['CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM'] = pis_sorted['CONTEST_OFFENSIVE_WIN_L4_CSUM']/pis_sorted['CONTEST_OFFENSIVE_L4_CSUM']*100
pis_sorted['CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM'] = pis_sorted['CONTEST_DEFENSIVE_LOSS_L4_CSUM']/pis_sorted['CONTEST_DEFENSIVE_L4_CSUM']*100

pis_sorted['CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM'] = pis_sorted['CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM'].fillna(0) 
pis_sorted['CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM'] = pis_sorted['CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM'].fillna(0)

In [16]:
pis_sorted = pis_sorted.drop(columns=['GOALS_CSUM', 'HITOUTS_CSUM', 'CONTEST_OFFENSIVE_CSUM', 'CONTEST_DEFENSIVE_CSUM',
                                     'GOALS_L4_CSUM', 'HITOUTS_L4_CSUM', 'CONTEST_OFFENSIVE_L4_CSUM', 'CONTEST_DEFENSIVE_L4_CSUM'])
pis_sorted.shape

(3652, 95)

In [17]:
pis_sorted['ROUND_NUMBER'] = pis_sorted.groupby(['SEASON', 'TEAM'])['ROUND_NUMBER'].shift(-1)

In [ ]:
pis_sorted

In [18]:
def check_missing_values(data):
    """
    Function to check for missing values in each column of a DataFrame.

    Parameters:
    - data: pandas DataFrame

    Returns:
    - missing_values: pandas Series containing the count of missing values in each column
    """
    missing_values = data.isnull().sum()
    return missing_values
for v in check_missing_values(pis_sorted):
    print(v)

0
162
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


# PROCESSING_Merging_Games_and_Ladder

In [19]:
# choose columns from ladder
cols_ladder = ['season','round_number','position', 'thisSeasonRecord.winLossRecord.wins',
               'form', 'team.abbreviation', 'thisSeasonRecord.percentage', 'pointsFor',
               'pointsAgainst']
ladder_short = ladder[cols_ladder]
ladder_short['ROUND_NUMBER_PLUS1'] = ladder_short['round_number'] + 1


# choose columns from games
games_cols = ['SEASON','ROUND_NUMBER','ROUND', 'VENUE','HOME_TEAM',
              'HOME_POINTS', 'HOME_WIN', 'HOME_LADDERLY_POSITION', 
              'AWAY_TEAM', 'AWAY_POINTS','AWAY_WIN', 'AWAY_LADDERLY_POSITION', 'DRAW']
games_short = games[games_cols]


# get last rounds from the ladder dataset
ladders_LASTROUND_rows = ladder_short.groupby('season')['round_number'].max()
ladder_short_LASTROUND = ladder_short[
    ladder_short.apply(lambda x: x['round_number'] == ladders_LASTROUND_rows[x['season']], axis=1)]

C:\Users\carlo\AppData\Local\Temp/ipykernel_21884/466636102.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ladder_short['ROUND_NUMBER_PLUS1'] = ladder_short['round_number'] + 1


In [ ]:
games_short

## Ladder

In [20]:
def form_cols(form):
    
    """
    Creates new columns based on form value.

    Parameters:
    - form (str): string value for a teams form
    
    Details:
    - Adds 'consecutive_wins': number of consecutive Ws starting from the end. 
    - Adds 'consecutive_losses': number of consecutive Ls starting from the end. ç
    - Adds 'lg_won': 1 if the end is a W, 0 if otherwise
    - Adds 'lg_won': number of Ws for the last 4 games, starting from the end.
    
    Returns:
    - consecutive_wins, consecutive_losses, lg_won, lg_won
    """

    consecutive_wins = 0
    for char in form[::-1]:
        if char == 'W':
            consecutive_wins += 1
        else:
            break
            
    consecutive_losses = 0
    for char in form[::-1]:
        if char == 'L':
            consecutive_losses += 1
        else:
            break
    lg_won = 1 if form[-1] == 'W' else 0
    l4g_wins = form[-4:].count('W') if len(form) >= 4 else form.count('W')
    # l5g_wins = form[-5:].count('W') if len(form) >= 5 else form.count('W')
    
    return consecutive_wins, consecutive_losses, lg_won, lg_won

In [21]:
def ladder_position(merge_try, casa_fuera, ladder_short_LASTROUND, last_ladder_position):
    
    '''
    Calculates ladder positions for each competing team. And fixes NA values
    
    Ladder position corresponds to the position BEFORE the game takes place, this is, 
    the position after the previous round is completed.
    
    Ladder position for the Finals series is taken to be that of the last completed 
    round in the Home-and-Away season
    
    Ladder position for the first round is taken to be that of the final round of the 
    Home-and-Away season of the prior season. Position is adjusted based on performance
    in the Finals season.
    
    Parameters:
        - merge_try: merged dataset
        - casa_fuera: 'HOME' or 'AWAY' to calculate the position for the HOME or AWAY team
        - ladder_short_LASTROUND: dataset processed form Ladders data with the ladder from 
        the last round in the home and away season. 
        - last_ladder_position: dataset with the ladder position from each team from the
        previous season
        
    Details:
        - Adds 'position': corresponding ladder position before the game takes place
        - Fills NAs for 'thisSeasonRecord.winLossRecord.wins': based on the Finals Series
        results.
        - Fills NAs for 'form': based on the Finals Series results.
        - Sets NAs in round 1 to 0
        - Adds 'CONSECUTIVE_WINS', 'CONSECUTIVE_LOSSES', 'LG_WON', 'L4G_WINS' applying 
        form_cols
        - Renames columns
    Returns:
        - merge_try: Merged dataset of games and ladders with newly created columns
        
    '''
    home_away_team = casa_fuera + '_TEAM'
    
    
    # Filling NAs Finals Series
    na_position_rows = merge_try[(merge_try['position'].isna()) & (merge_try['ROUND_NUMBER'] != 1)]
    
    for index, row in na_position_rows.iterrows():
        
        # position
        season = row['SEASON']
        team = row[home_away_team]
        round_number = row['ROUND_NUMBER']
        prev_round_number = round_number - 1
        corresponding_position = ladder_short_LASTROUND[(ladder_short_LASTROUND['season'] == season)\
                                                        & (ladder_short_LASTROUND['team.abbreviation'] == team)
                                                       ]['position'].values[0]
        # print(season,team,corresponding_position)
        merge_try.loc[index, 'position'] = corresponding_position
        
        # cummulative wins
        corresponding_cummulative_wins = ladder_short_LASTROUND[(ladder_short_LASTROUND['season'] == season)\
                                                        & (ladder_short_LASTROUND['team.abbreviation'] == team)
                                                       ]['thisSeasonRecord.winLossRecord.wins'].values[0]
        
        if merge_try.loc[index, 'ROUND'] == 'SF':
            if corresponding_position >= 5:
                merge_try.loc[index, 'thisSeasonRecord.winLossRecord.wins'] = corresponding_cummulative_wins + 1
            else:
                merge_try.loc[index, 'thisSeasonRecord.winLossRecord.wins'] = corresponding_cummulative_wins
        elif merge_try.loc[index, 'ROUND'] == 'PF':
            if corresponding_position >= 5:
                merge_try.loc[index, 'thisSeasonRecord.winLossRecord.wins'] = corresponding_cummulative_wins + 2
            else:
                merge_try.loc[index, 'thisSeasonRecord.winLossRecord.wins'] = corresponding_cummulative_wins + 1
        elif merge_try.loc[index, 'ROUND'] == 'GF':
            if corresponding_position >= 5:
                merge_try.loc[index, 'thisSeasonRecord.winLossRecord.wins'] = corresponding_cummulative_wins + 3    
            else:
                merge_try.loc[index, 'thisSeasonRecord.winLossRecord.wins'] = corresponding_cummulative_wins + 2
        
        #form
        
        corresponding_form = ladder_short_LASTROUND[(ladder_short_LASTROUND['season'] == season)\
                                                        & (ladder_short_LASTROUND['team.abbreviation'] == team)
                                                       ]['form'].values[0]
        mergi_semi_HOME = merge_try[(merge_try['ROUND'] == 'SF') & (merge_try['SEASON'] == season)]['HOME_TEAM'].tolist()
        mergi_semi_AWAY = merge_try[(merge_try['ROUND'] == 'SF') & (merge_try['SEASON'] == season)]['AWAY_TEAM'].tolist()
        
        if merge_try.loc[index, 'ROUND'] == 'SF':
            if corresponding_position >= 5:
                merge_try.loc[index, 'form'] = corresponding_form + 'W'
            elif corresponding_position < 5:
                merge_try.loc[index, 'form'] = corresponding_form + 'L'
                
        elif merge_try.loc[index, 'ROUND'] == 'PF':
            if corresponding_position >= 5:
                merge_try.loc[index, 'form'] = corresponding_form + 'WW'
            elif corresponding_position < 5:
                # print(f"{season}\n{team}\n{mergi_semi_HOME}\n{mergi_semi_AWAY}")
                if team in mergi_semi_HOME or team in mergi_semi_AWAY:
                    # print('oop')
                    merge_try.loc[index, 'form'] = corresponding_form + 'LW'
                else:
                    merge_try.loc[index, 'form'] = corresponding_form + 'W'
                    
        elif merge_try.loc[index, 'ROUND'] == 'GF':
            if corresponding_position >= 5:
                merge_try.loc[index, 'form'] = corresponding_form + 'WWW' 
            elif corresponding_position < 5:
                if team in mergi_semi_HOME or team in mergi_semi_AWAY:
                    merge_try.loc[index, 'form'] = corresponding_form + 'LWW'
                else:
                    merge_try.loc[index, 'form'] = corresponding_form + 'WW'
        
    
    # Filling NAs ROUND 1

    na_position_rows1 = merge_try[(merge_try['position'].isna()) & (merge_try['ROUND_NUMBER'] == 1)]

    for index, row in na_position_rows1.iterrows():
        
        # position
            
        merge_try.loc[index, 'position'] = merge_try.loc[index, casa_fuera +'_LADDERLY_POSITION']
        merge_try.loc[index, 'thisSeasonRecord.winLossRecord.wins'] = 0
        merge_try.loc[index, 'form'] = 'U'
        merge_try.loc[index, 'thisSeasonRecord.percentage'] = 0
        merge_try.loc[index, 'pointsFor'] = 0
        merge_try.loc[index, 'pointsAgainst'] = 0

    
    # New columns derived from FORM
    merge_try[['CONSECUTIVE_WINS', 'CONSECUTIVE_LOSSES', 'LG_WON', 'L4G_WINS']] = pd.DataFrame(
        merge_try['form'].apply(lambda x: form_cols(x)).tolist(),
        columns=['CONSECUTIVE_WINS', 'CONSECUTIVE_LOSSES', 'LG_WON', 'L4G_WINS'])
    
    #ASSERTING NO ERRORS 

    assert (merge_try[merge_try['ROUND_NUMBER'] == 1][
        'position'] == merge_try[merge_try['ROUND_NUMBER'] == 1
                                ][casa_fuera + '_LADDERLY_POSITION']).all(), "Position Wrong at Round 1"
    merge = merge_try.dropna(subset=['season', 'round_number', 'team.abbreviation'])
    assert (merge['SEASON'] == merge['season'].astype(int)).all(), "Season columns are not the same after conversion to int."
    assert (merge['ROUND_NUMBER'] == merge['round_number']\
            .astype(int)+1).all(), "round columns are not the same after conversion to int."
    assert (merge[home_away_team] == merge['team.abbreviation'])\
        .all(), "Team columns are not the same."

    # Dropping and Renaming
    merge_try = merge_try.drop(columns=['season', 'round_number', 'team.abbreviation',
                                        'form', 'ROUND_NUMBER_PLUS1'])
    merge_try = merge_try.rename(columns={'position': f"{casa_fuera}_LADDER_POSITION"})
    merge_try = merge_try.rename(columns={'thisSeasonRecord.winLossRecord.wins': f"{casa_fuera}_WINS_CUMMULATIVE"})
    # merge_try = merge_try.rename(columns={'form': f"{casa_fuera}_FORM"})
    merge_try = merge_try.rename(columns={'CONSECUTIVE_WINS': f"{casa_fuera}_CONSECUTIVE_WINS"})
    merge_try = merge_try.rename(columns={'CONSECUTIVE_LOSSES': f"{casa_fuera}_CONSECUTIVE_LOSSES"})
    merge_try = merge_try.rename(columns={'LG_WON': f"{casa_fuera}_LG_WON"})
    merge_try = merge_try.rename(columns={'L4G_WINS': f"{casa_fuera}_L4G_WINS"})
    # merge_try = merge_try.rename(columns={'L5G_WINS': f"{casa_fuera}_L5G_WINS"})
    # merge_try = merge_try.rename(columns={'L5G_WINS': f"{casa_fuera}_L5G_WINS"})
    merge_try = merge_try.rename(columns={'pointsFor': f"{casa_fuera}_POINTSFOR"})
    merge_try = merge_try.rename(columns={'pointsAgainst': f"{casa_fuera}_POINTSAGAINST"})
    merge_try = merge_try.rename(columns={'thisSeasonRecord.percentage': f"{casa_fuera}_PERCENTAGE"})
    
    return merge_try

## HOME

#### Merging + Run Processing

In [22]:
# merging HOME
merge_home = pd.merge(games_short, ladder_short, 
         left_on=['SEASON','ROUND_NUMBER', 'HOME_TEAM'], right_on=['season','ROUND_NUMBER_PLUS1','team.abbreviation'], 
         how='left')

merge_home = ladder_position(merge_home, 'HOME', ladder_short_LASTROUND, last_ladder_position)
merge_home

,SEASON,ROUND_NUMBER,ROUND,VENUE,HOME_TEAM,HOME_POINTS,HOME_WIN,HOME_LADDERLY_POSITION,AWAY_TEAM,AWAY_POINTS,...,DRAW,HOME_LADDER_POSITION,HOME_WINS_CUMMULATIVE,HOME_PERCENTAGE,HOME_POINTSFOR,HOME_POINTSAGAINST,HOME_CONSECUTIVE_WINS,HOME_CONSECUTIVE_LOSSES,HOME_LG_WON,HOME_L4G_WINS
0,2015,1,R1,M.C.G.,CARL,78,0,13,RICH,105,...,0,13.0,0.0,0.0,0.0,0.0,0,0,0,0
1,2015,1,R1,M.C.G.,MELB,115,1,17,GCFC,89,...,0,17.0,0.0,0.0,0.0,0.0,0,0,0,0
2,2015,1,R1,Stadium Australia,SYD,72,1,2,ESS,60,...,0,2.0,0.0,0.0,0.0,0.0,0,0,0,0
3,2015,1,R1,Gabba,BL,74,0,15,COLL,86,...,0,15.0,0.0,0.0,0.0,0.0,0,0,0,0
4,2015,1,R1,Docklands,WB,97,1,14,WCE,87,...,0,14.0,0.0,0.0,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2023,26,SF,M.C.G.,MELB,71,0,5,CARL,73,...,0,4.0,16.0,NaN,NaN,NaN,0,1,0,2
1822,2023,26,SF,Adelaide Oval,PORT,70,0,11,GWS,93,...,0,3.0,17.0,NaN,NaN,NaN,0,1,0,3
1823,2023,27,PF,M.C.G.,COLL,58,1,3,GWS,57,...,0,1.0,19.0,NaN,NaN,NaN,2,0,1,3
1824,2023,27,PF,Gabba,BL,79,1,4,CARL,63,...,0,2.0,18.0,NaN,NaN,NaN,5,0,1,4


## AWAY

### Merge + Run Processing

In [23]:
def nas_pointsfor(merge_away):

    '''
    Fills NAs for POINTSFOR and POINTSGAINST.
    
    Parameters:
        - merge_away: merged dataset
        
    Details:
        - Fills NAs for 'POINTSFOR', 'POINTSAGAINST': based on the Finals Series
        results.
        - Adds 'PERCENTAGE' = POINTSFOR/POINTSAGAINST
        
    Returns:
        - merge_away: Merged dataset of games and ladders with newly created columns
        
    '''
    
    na_position_rows = merge_away[(merge_away['HOME_POINTSFOR'].isna())]
    
    for index, row in na_position_rows.iterrows():
        
        season = row['SEASON']
        home_team = row['HOME_TEAM']
        away_team = row['AWAY_TEAM']
        round_number = row['ROUND_NUMBER']
        home_prev_round_number = round_number - 1
        away_prev_round_number = round_number - 1
        home_position = row['HOME_LADDER_POSITION']
        away_position = row['AWAY_LADDER_POSITION']
        index_prev = None
        
        mergi_semi_HOME = merge_away[(merge_away['ROUND'] == 'SF') & (merge_away['SEASON'] == season)]['HOME_TEAM'].tolist()
        mergi_semi_AWAY = merge_away[(merge_away['ROUND'] == 'SF') & (merge_away['SEASON'] == season)]['AWAY_TEAM'].tolist()    
    
        
        if (home_position < 5 and 
            home_team not in mergi_semi_HOME and
            home_team not in mergi_semi_AWAY and 
            merge_away.loc[index, 'ROUND'] == 'PF'):
                home_prev_round_number = home_prev_round_number - 1
                
                
        home_mergi_prevround_HOME = merge_away[(merge_away['ROUND_NUMBER'] == home_prev_round_number)\
                                         & (merge_away['SEASON'] == season) 
                                        ]['HOME_TEAM'].tolist()
        home_mergi_prevround_AWAY = merge_away[(merge_away['ROUND_NUMBER'] == home_prev_round_number)\
                                         & (merge_away['SEASON'] == season)
                                        ]['AWAY_TEAM'].tolist()       
        if home_team in home_mergi_prevround_HOME:
            index_prev = merge_away[(merge_away['ROUND_NUMBER'] == home_prev_round_number) &
                       (merge_away['SEASON'] == season) &
                       (merge_away['HOME_TEAM'] == home_team)].index
            # print(season, home_team,mergi_prevround_HOME, index_prev)
            merge_away.loc[index, 'HOME_POINTSFOR'] = merge_away.loc[index_prev, 'HOME_POINTSFOR'].values[0] + \
                                             merge_away.loc[index_prev, 'HOME_POINTS'].values[0]
            merge_away.loc[index, 'HOME_POINTSAGAINST'] = merge_away.loc[index_prev, 'HOME_POINTSAGAINST'].values[0] + \
                                             merge_away.loc[index_prev, 'AWAY_POINTS'].values[0]
        elif home_team in home_mergi_prevround_AWAY:
            index_prev = merge_away[(merge_away['ROUND_NUMBER'] == home_prev_round_number) &
                       (merge_away['SEASON'] == season) &
                       (merge_away['AWAY_TEAM'] == home_team)].index
            # print(season, home_team,mergi_prevround_AWAY, index_prev)
            merge_away.loc[index, 'HOME_POINTSAGAINST'] = merge_away.loc[index_prev, 'AWAY_POINTSAGAINST'].values[0] + \
                                             merge_away.loc[index_prev, 'HOME_POINTS'].values[0]
            merge_away.loc[index, 'HOME_POINTSFOR'] = merge_away.loc[index_prev, 'AWAY_POINTSFOR'].values[0] + \
                                             merge_away.loc[index_prev, 'AWAY_POINTS'].values[0]

        away_mergi_prevround_HOME = merge_away[(merge_away['ROUND_NUMBER'] == away_prev_round_number)\
                                         & (merge_away['SEASON'] == season) 
                                        ]['HOME_TEAM'].tolist()
        away_mergi_prevround_AWAY = merge_away[(merge_away['ROUND_NUMBER'] == away_prev_round_number)\
                                         & (merge_away['SEASON'] == season)
                                        ]['AWAY_TEAM'].tolist()             
               
        if away_team in away_mergi_prevround_HOME:
            # print(season, away_prev_round_number, away_team)
            index_prev = merge_away[(merge_away['ROUND_NUMBER'] == away_prev_round_number) &
                       (merge_away['SEASON'] == season) &
                       (merge_away['HOME_TEAM'] == away_team)].index
            merge_away.loc[index, 'AWAY_POINTSFOR'] = merge_away.loc[index_prev, 'HOME_POINTSFOR'].values[0] + \
                                             merge_away.loc[index_prev, 'HOME_POINTS'].values[0]
            merge_away.loc[index, 'AWAY_POINTSAGAINST'] = merge_away.loc[index_prev, 'HOME_POINTSAGAINST'].values[0] + \
                                             merge_away.loc[index_prev, 'AWAY_POINTS'].values[0]
        elif away_team in away_mergi_prevround_AWAY:
            index_prev = merge_away[(merge_away['ROUND_NUMBER'] == away_prev_round_number) &
                                    (merge_away['SEASON'] == season) &
                                    (merge_away['AWAY_TEAM'] == away_team)].index
            merge_away.loc[index, 'AWAY_POINTSFOR'] = merge_away.loc[index_prev, 'AWAY_POINTSFOR'].values[0] + \
                                             merge_away.loc[index_prev, 'AWAY_POINTS'].values[0]
            merge_away.loc[index, 'AWAY_POINTSAGAINST'] = merge_away.loc[index_prev, 'AWAY_POINTSAGAINST'].values[0] + \
                                             merge_away.loc[index_prev, 'HOME_POINTS'].values[0]              
            
        merge_away.loc[index, 'HOME_PERCENTAGE'] = merge_away.loc[index, 'HOME_POINTSFOR']/merge_away.loc[index, 'HOME_POINTSAGAINST']*100
        merge_away.loc[index, 'AWAY_PERCENTAGE'] = merge_away.loc[index, 'AWAY_POINTSFOR']/merge_away.loc[index, 'AWAY_POINTSAGAINST']*100
    return merge_away

In [24]:
# merging AWAY
merge_away = pd.merge(merge_home, ladder_short, 
         left_on=['SEASON','ROUND_NUMBER', 'AWAY_TEAM'], right_on=['season','ROUND_NUMBER_PLUS1','team.abbreviation'], 
         how='left')
merge_away = ladder_position(merge_away, 'AWAY', ladder_short_LASTROUND, last_ladder_position)
merge_away = nas_pointsfor(merge_away)
merge_away

,SEASON,ROUND_NUMBER,ROUND,VENUE,HOME_TEAM,HOME_POINTS,HOME_WIN,HOME_LADDERLY_POSITION,AWAY_TEAM,AWAY_POINTS,...,HOME_L4G_WINS,AWAY_LADDER_POSITION,AWAY_WINS_CUMMULATIVE,AWAY_PERCENTAGE,AWAY_POINTSFOR,AWAY_POINTSAGAINST,AWAY_CONSECUTIVE_WINS,AWAY_CONSECUTIVE_LOSSES,AWAY_LG_WON,AWAY_L4G_WINS
0,2015,1,R1,M.C.G.,CARL,78,0,13,RICH,105,...,0,7.0,0.0,0.000000,0.0,0.0,0,0,0,0
1,2015,1,R1,M.C.G.,MELB,115,1,17,GCFC,89,...,0,12.0,0.0,0.000000,0.0,0.0,0,0,0,0
2,2015,1,R1,Stadium Australia,SYD,72,1,2,ESS,60,...,0,8.0,0.0,0.000000,0.0,0.0,0,0,0,0
3,2015,1,R1,Gabba,BL,74,0,15,COLL,86,...,0,11.0,0.0,0.000000,0.0,0.0,0,0,0,0
4,2015,1,R1,Docklands,WB,97,1,14,WCE,87,...,0,9.0,0.0,0.000000,0.0,0.0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2023,26,SF,M.C.G.,MELB,71,0,5,CARL,73,...,2,5.0,14.0,113.087819,1996.0,1765.0,1,0,1,3
1822,2023,26,SF,Adelaide Oval,PORT,70,0,11,GWS,93,...,3,7.0,14.0,108.002039,2119.0,1962.0,3,0,1,3
1823,2023,27,PF,M.C.G.,COLL,58,1,3,GWS,57,...,3,7.0,15.0,108.858268,2212.0,2032.0,4,0,1,4
1824,2023,27,PF,Gabba,BL,79,1,4,CARL,63,...,4,5.0,15.0,112.690632,2069.0,1836.0,2,0,1,3


# Merging with PIs

In [26]:
def merge_with_PIs(results_ladder, pis, casa_fuera):
    
    '''
    Merges games and ladders data with PIs data.
    
    Parameters:
        - results_ladder: dataset with game results and ladder data.
        - PIs: dataset with PIs data
        - casa_fuera (str): string indicating whether we want to merge home or away.
        
    Returns:
        - merged: Merged dataset with newly created columns
        
    '''
    
    print(results_ladder.shape, pis.shape)
    merged = pd.merge(results_ladder, pis, left_on=['SEASON','ROUND_NUMBER', f'{casa_fuera}_TEAM'],
                      right_on=['SEASON','ROUND_NUMBER', 'TEAM'], how='left')
    for column in merged.columns:
        if column in pis.columns and column not in ['SEASON','ROUND_NUMBER']:
            merged.rename(columns = {column: f'{casa_fuera}_{column}'}, inplace = True)
    merged.fillna(0, inplace=True)
    merged = merged.loc[:,~merged.columns.duplicated()]
    print(merged.shape)
    return merged

In [27]:
merge_pis_home = merge_with_PIs(merge_away, pis_sorted, 'HOME')

(1826, 31) (3652, 95)
(1826, 123)


In [31]:
merge_pis_complete = merge_with_PIs(merge_pis_home, pis_sorted, 'AWAY')
merge_pis_complete

(1826, 123) (3652, 95)
(1826, 215)


,SEASON,ROUND_NUMBER,ROUND,VENUE,HOME_TEAM,HOME_POINTS,HOME_WIN,HOME_LADDERLY_POSITION,AWAY_TEAM,AWAY_POINTS,...,AWAY_CONTEST_OFFENSIVE_WIN_RATE_CSUM,AWAY_CONTEST_DEFENSIVE_LOSS_RATE_CSUM,AWAY_DISPOSALS_EFFICIENCY_L4_CSUM,AWAY_KICKS_EFFICIENCY_L4_CSUM,AWAY_KICK2HANDBALL_L4_CSUM,AWAY_POSESSIONS_CONTESTED_RATE_L4_CSUM,AWAY_GOALS_ACCURACY_L4_CSUM,AWAY_HITOUTS_ADVANTAGE_RATE_L4_CSUM,AWAY_CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM,AWAY_CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM
0,2015,1,R1,M.C.G.,CARL,78,0,13,RICH,105,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,2015,1,R1,M.C.G.,MELB,115,1,17,GCFC,89,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,2015,1,R1,Stadium Australia,SYD,72,1,2,ESS,60,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2015,1,R1,Gabba,BL,74,0,15,COLL,86,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,2015,1,R1,Docklands,WB,97,1,14,WCE,87,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2023,26,SF,M.C.G.,MELB,71,0,5,CARL,73,...,28.720627,24.437299,71.776316,64.088398,1.471545,42.187500,52.380952,23.021583,28.378378,20.833333
1822,2023,26,SF,Adelaide Oval,PORT,70,0,11,GWS,93,...,20.819113,21.975309,73.432450,66.081871,1.235549,37.360473,56.557377,33.587786,23.404255,25.000000
1823,2023,27,PF,M.C.G.,COLL,58,1,3,GWS,57,...,20.588235,21.545667,72.981564,65.597345,1.351271,37.475602,52.671756,32.352941,18.367347,23.880597
1824,2023,27,PF,Gabba,BL,79,1,4,CARL,63,...,28.426396,24.534161,71.638796,64.212911,1.442810,41.982106,51.111111,25.179856,29.824561,25.000000


In [32]:
merge_pis_complete[['HOME_LADDERLY_POSITION', 'AWAY_LADDERLY_POSITION']]

,HOME_LADDERLY_POSITION,AWAY_LADDERLY_POSITION
0,13,7
1,17,12
2,2,8
3,15,11
4,14,9
...,...,...
1821,5,9
1822,11,16
1823,3,16
1824,4,9


# PROCESSING_All_Variables_Home_Away_Ready

## Functions 

In [33]:
def difurentials(merge_away):
    
    '''
    Processes columns as differentials as specified in the thesis.
        
    '''
    
    data = merge_away.copy(deep = True)
    column_pairs1 = [
        ('AWAY_LADDER_POSITION', 'HOME_LADDER_POSITION'),
        ('AWAY_LADDERLY_POSITION', 'HOME_LADDERLY_POSITION'),
        ('AWAY_POINTSAGAINST', 'HOME_POINTSAGAINST'),
        ('AWAY_CLANGERS_CSUM','HOME_CLANGERS_CSUM'),
        ('AWAY_CLANGERS_L4_CSUM','HOME_CLANGERS_L4_CSUM'),
        ('AWAY_FREES_AGAINST_CSUM','HOME_FREES_AGAINST_CSUM'),
        ('AWAY_FREES_AGAINST_L4_CSUM','HOME_FREES_AGAINST_L4_CSUM'),
        ('AWAY_TURNOVERS_CSUM','HOME_TURNOVERS_CSUM'),
        ('AWAY_TURNOVERS_L4_CSUM','HOME_TURNOVERS_L4_CSUM'),
        ('AWAY_CONTEST_DEFENSIVE_LOSS_CSUM','HOME_CONTEST_DEFENSIVE_LOSS_CSUM'),
        ('AWAY_CONTEST_DEFENSIVE_LOSS_L4_CSUM','HOME_CONTEST_DEFENSIVE_LOSS_L4_CSUM'),
    ]
    column_pairs = [
        ('AWAY_WINS_CUMMULATIVE', 'HOME_WINS_CUMMULATIVE'),
        ('AWAY_PERCENTAGE', 'HOME_PERCENTAGE'),
        ('AWAY_POINTSFOR', 'HOME_POINTSFOR'),
        ('AWAY_DISPOSALS_CSUM','HOME_DISPOSALS_CSUM'),
        ('AWAY_DISPOSALS_L4_CSUM','HOME_DISPOSALS_L4_CSUM'),
        ('AWAY_DISPOSALS_EFFECTIVE_CSUM','HOME_DISPOSALS_EFFECTIVE_CSUM'),
        ('AWAY_DISPOSALS_EFFECTIVE_L4_CSUM','HOME_DISPOSALS_EFFECTIVE_L4_CSUM'),
        ('AWAY_KICKS_CSUM','HOME_KICKS_CSUM'),
        ('AWAY_KICKS_L4_CSUM','HOME_KICKS_L4_CSUM'),
        ('AWAY_KICKS_EFFECTIVE_CSUM','HOME_KICKS_EFFECTIVE_CSUM'),
        ('AWAY_KICKS_EFFECTIVE_L4_CSUM','HOME_KICKS_EFFECTIVE_L4_CSUM'),
        ('AWAY_HANDBALLS_CSUM','HOME_HANDBALLS_CSUM'),
        ('AWAY_HANDBALLS_L4_CSUM','HOME_HANDBALLS_L4_CSUM'),
        ('AWAY_MARKS_CSUM','HOME_MARKS_CSUM'),
        ('AWAY_MARKS_L4_CSUM','HOME_MARKS_L4_CSUM'),
        ('AWAY_MARKS_INSIDE50_CSUM','HOME_MARKS_INSIDE50_CSUM'),
        ('AWAY_MARKS_INSIDE50_L4_CSUM','HOME_MARKS_INSIDE50_L4_CSUM'),
        ('AWAY_MARKS_CONTESTED_CSUM','HOME_MARKS_CONTESTED_CSUM'),
        ('AWAY_MARKS_CONTESTED_L4_CSUM','HOME_MARKS_CONTESTED_L4_CSUM'),
        ('AWAY_MARKS_ONLEAD_CSUM','HOME_MARKS_ONLEAD_CSUM'),
        ('AWAY_MARKS_ONLEAD_L4_CSUM','HOME_MARKS_ONLEAD_L4_CSUM'),
        ('AWAY_MARKS_INTERCEPT_CSUM','HOME_MARKS_INTERCEPT_CSUM'),
        ('AWAY_MARKS_INTERCEPT_L4_CSUM','HOME_MARKS_INTERCEPT_L4_CSUM'),
        ('AWAY_BOUNCES_CSUM','HOME_BOUNCES_CSUM'),
        ('AWAY_BOUNCES_L4_CSUM','HOME_BOUNCES_L4_CSUM'),
        ('AWAY_METRES_GAINED_CSUM','HOME_METRES_GAINED_CSUM'),
        ('AWAY_METRES_GAINED_L4_CSUM','HOME_METRES_GAINED_L4_CSUM'),
        ('AWAY_POSESSIONS_CSUM','HOME_POSESSIONS_CSUM'),
        ('AWAY_POSESSIONS_L4_CSUM','HOME_POSESSIONS_L4_CSUM'),
        ('AWAY_POSESSIONS_CONTESTED_CSUM','HOME_POSESSIONS_CONTESTED_CSUM'),
        ('AWAY_POSESSIONS_CONTESTED_L4_CSUM','HOME_POSESSIONS_CONTESTED_L4_CSUM'),
        ('AWAY_POSESSIONS_UNCONTESTED_CSUM','HOME_POSESSIONS_UNCONTESTED_CSUM'),
        ('AWAY_POSESSIONS_UNCONTESTED_L4_CSUM','HOME_POSESSIONS_UNCONTESTED_L4_CSUM'),
        ('AWAY_GETS_GROUNDBALL_CSUM','HOME_GETS_GROUNDBALL_CSUM'),
        ('AWAY_GETS_GROUNDBALL_L4_CSUM','HOME_GETS_GROUNDBALL_L4_CSUM'),
        ('AWAY_GETS_GROUNDBALL50_CSUM','HOME_GETS_GROUNDBALL50_CSUM'),
        ('AWAY_GETS_GROUNDBALL50_L4_CSUM','HOME_GETS_GROUNDBALL50_L4_CSUM'),
        ('AWAY_INSIDE50_CSUM','HOME_INSIDE50_CSUM'),
        ('AWAY_INSIDE50_L4_CSUM','HOME_INSIDE50_L4_CSUM'),
        ('AWAY_GOALS_SHOTS_CSUM','HOME_GOALS_SHOTS_CSUM'),
        ('AWAY_GOALS_SHOTS_L4_CSUM','HOME_GOALS_SHOTS_L4_CSUM'),
        ('AWAY_SCORE_LAUNCHES_CSUM','HOME_SCORE_LAUNCHES_CSUM'),
        ('AWAY_SCORE_LAUNCHES_L4_CSUM','HOME_SCORE_LAUNCHES_L4_CSUM'),
        ('AWAY_CLEARANCES_CSUM','HOME_CLEARANCES_CSUM'),
        ('AWAY_CLEARANCES_L4_CSUM','HOME_CLEARANCES_L4_CSUM'),
        ('AWAY_CLEARANCES_CENTRE_CSUM','HOME_CLEARANCES_CENTRE_CSUM'),
        ('AWAY_CLEARANCES_CENTRE_L4_CSUM','HOME_CLEARANCES_CENTRE_L4_CSUM'),
        ('AWAY_CLEARANCES_STOPPAGE_CSUM','HOME_CLEARANCES_STOPPAGE_CSUM'),
        ('AWAY_CLEARANCES_STOPPAGE_L4_CSUM','HOME_CLEARANCES_STOPPAGE_L4_CSUM'),
        ('AWAY_HITOUTS_ADVANTAGE_CSUM','HOME_HITOUTS_ADVANTAGE_CSUM'),
        ('AWAY_HITOUTS_ADVANTAGE_L4_CSUM','HOME_HITOUTS_ADVANTAGE_L4_CSUM'),
        ('AWAY_HITOUTS_WIN_RATE_CSUM','HOME_HITOUTS_WIN_RATE_CSUM'),
        ('AWAY_HITOUTS_WIN_RATE_L4_CSUM','HOME_HITOUTS_WIN_RATE_L4_CSUM'),
        ('AWAY_CONTEST_OFFENSIVE_WIN_CSUM','HOME_CONTEST_OFFENSIVE_WIN_CSUM'),
        ('AWAY_CONTEST_OFFENSIVE_WIN_L4_CSUM','HOME_CONTEST_OFFENSIVE_WIN_L4_CSUM'),
        ('AWAY_TACKLES_CSUM','HOME_TACKLES_CSUM'),
        ('AWAY_TACKLES_L4_CSUM','HOME_TACKLES_L4_CSUM'),
        ('AWAY_TACKLES_INSIDE50_CSUM','HOME_TACKLES_INSIDE50_CSUM'),
        ('AWAY_TACKLES_INSIDE50_L4_CSUM','HOME_TACKLES_INSIDE50_L4_CSUM'),
        ('AWAY_ONE_PERCENTERS_CSUM','HOME_ONE_PERCENTERS_CSUM'),
        ('AWAY_ONE_PERCENTERS_L4_CSUM','HOME_ONE_PERCENTERS_L4_CSUM'),
        ('AWAY_REBOUND_INSIDE50S_CSUM','HOME_REBOUND_INSIDE50S_CSUM'),
        ('AWAY_REBOUND_INSIDE50S_L4_CSUM','HOME_REBOUND_INSIDE50S_L4_CSUM'),
        ('AWAY_INTERCEPTS_CSUM','HOME_INTERCEPTS_CSUM'),
        ('AWAY_INTERCEPTS_L4_CSUM','HOME_INTERCEPTS_L4_CSUM'),
        ('AWAY_PRESSURE_CSUM','HOME_PRESSURE_CSUM'),
        ('AWAY_PRESSURE_L4_CSUM','HOME_PRESSURE_L4_CSUM'),
        ('AWAY_PRESSURE_DEFENSEHALF_CSUM','HOME_PRESSURE_DEFENSEHALF_CSUM'),
        ('AWAY_PRESSURE_DEFENSEHALF_L4_CSUM','HOME_PRESSURE_DEFENSEHALF_L4_CSUM'),
        ('AWAY_SPOILS_CSUM','HOME_SPOILS_CSUM'),
        ('AWAY_SPOILS_L4_CSUM','HOME_SPOILS_L4_CSUM'),
        ('AWAY_DISPOSALS_EFFICIENCY_CSUM','HOME_DISPOSALS_EFFICIENCY_CSUM'),
        ('AWAY_DISPOSALS_EFFICIENCY_L4_CSUM','HOME_DISPOSALS_EFFICIENCY_L4_CSUM'),
        ('AWAY_KICKS_EFFICIENCY_CSUM','HOME_KICKS_EFFICIENCY_CSUM'),
        ('AWAY_KICKS_EFFICIENCY_L4_CSUM','HOME_KICKS_EFFICIENCY_L4_CSUM'),
        ('AWAY_KICK2HANDBALL_CSUM','HOME_KICK2HANDBALL_CSUM'),
        ('AWAY_KICK2HANDBALL_L4_CSUM','HOME_KICK2HANDBALL_L4_CSUM'),
        ('AWAY_POSESSIONS_CONTESTED_RATE_CSUM','HOME_POSESSIONS_CONTESTED_RATE_CSUM'),
        ('AWAY_POSESSIONS_CONTESTED_RATE_L4_CSUM','HOME_POSESSIONS_CONTESTED_RATE_L4_CSUM'),
        ('AWAY_GOALS_ACCURACY_CSUM','HOME_GOALS_ACCURACY_CSUM'),
        ('AWAY_GOALS_ACCURACY_L4_CSUM','HOME_GOALS_ACCURACY_L4_CSUM'),
        ('AWAY_HITOUTS_ADVANTAGE_RATE_CSUM','HOME_HITOUTS_ADVANTAGE_RATE_CSUM'),
        ('AWAY_HITOUTS_ADVANTAGE_RATE_L4_CSUM','HOME_HITOUTS_ADVANTAGE_RATE_L4_CSUM'),
        ('AWAY_CONTEST_OFFENSIVE_WIN_RATE_CSUM','HOME_CONTEST_OFFENSIVE_WIN_RATE_CSUM'),
        ('AWAY_CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM','HOME_CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM'),
        ('AWAY_CONTEST_DEFENSIVE_LOSS_RATE_CSUM','HOME_CONTEST_DEFENSIVE_LOSS_RATE_CSUM'),
        ('AWAY_CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM','HOME_CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM')    
        
    ]
    
    for away_col, home_col in column_pairs1:
        data[f'{home_col}_DIFF'] = data[away_col] - data[home_col]
        data[f'{away_col}_DIFF'] = -data[f'{home_col}_DIFF']
    
    for away_col, home_col in column_pairs:
        data[f'{home_col}_DIFF'] = data[home_col] - data[away_col] 
        data[f'{away_col}_DIFF'] = -data[f'{home_col}_DIFF']
    
    columns_to_drop = [col for col_pair in column_pairs for col in col_pair] + \
                      [col for col_pair in column_pairs1 for col in col_pair]
    data = data.drop(columns=columns_to_drop)
    
    return data

## Run Processing

In [34]:
resultss_before_modelling = difurentials(merge_pis_complete)
resultss_before_modelling

C:\Users\carlo\AppData\Local\Temp/ipykernel_21884/2225191984.py:114: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{away_col}_DIFF'] = -data[f'{home_col}_DIFF']
C:\Users\carlo\AppData\Local\Temp/ipykernel_21884/2225191984.py:113: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  data[f'{home_col}_DIFF'] = data[home_col] - data[away_col]


,SEASON,ROUND_NUMBER,ROUND,VENUE,HOME_TEAM,HOME_POINTS,HOME_WIN,AWAY_TEAM,AWAY_POINTS,AWAY_WIN,...,HOME_HITOUTS_ADVANTAGE_RATE_L4_CSUM_DIFF,AWAY_HITOUTS_ADVANTAGE_RATE_L4_CSUM_DIFF,HOME_CONTEST_OFFENSIVE_WIN_RATE_CSUM_DIFF,AWAY_CONTEST_OFFENSIVE_WIN_RATE_CSUM_DIFF,HOME_CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM_DIFF,AWAY_CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM_DIFF,HOME_CONTEST_DEFENSIVE_LOSS_RATE_CSUM_DIFF,AWAY_CONTEST_DEFENSIVE_LOSS_RATE_CSUM_DIFF,HOME_CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM_DIFF,AWAY_CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM_DIFF
0,2015,1,R1,M.C.G.,CARL,78,0,RICH,105,1,...,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000
1,2015,1,R1,M.C.G.,MELB,115,1,GCFC,89,0,...,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000
2,2015,1,R1,Stadium Australia,SYD,72,1,ESS,60,0,...,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000
3,2015,1,R1,Gabba,BL,74,0,COLL,86,1,...,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000
4,2015,1,R1,Docklands,WB,97,1,WCE,87,0,...,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000,0.000000,-0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1821,2023,26,SF,M.C.G.,MELB,71,0,CARL,73,1,...,0.396139,-0.396139,-6.212588,6.212588,4.179761,-4.179761,-4.956780,4.956780,-2.261905,2.261905
1822,2023,26,SF,Adelaide Oval,PORT,70,0,GWS,93,1,...,-3.490699,3.490699,5.966602,-5.966602,2.825253,-2.825253,10.386207,-10.386207,8.962264,-8.962264
1823,2023,27,PF,M.C.G.,COLL,58,1,GWS,57,0,...,-8.400845,8.400845,6.368286,-6.368286,0.500578,-0.500578,2.006456,-2.006456,8.677543,-8.677543
1824,2023,27,PF,Gabba,BL,79,1,CARL,63,0,...,-3.846523,3.846523,0.013971,-0.013971,6.713900,-6.713900,-4.468372,4.468372,-0.862069,0.862069


In [ ]:
resultss_before_modelling.columns

# PROCESSINGPrep4Modelling

### Functions

In [36]:
venues_loc = {
    'VIC' : ['M.C.G.', 'Docklands', 'Kardinia Park', 'Eureka Stadium'],
    'SA' : ['Adelaide Oval', 'Norwood Oval', 'Football Park', 'Summit Sports Park'],
    'NSW' : ['Sydney Showground', 'S.C.G.', 'Stadium Australia', 'Blacktown'],
    'QLD' : ['Gabba', 'Carrara', 'Cazaly\'s Stadium', 'Riverway Stadium'],
    'WA' : ['Perth Stadium', 'Subiaco'],
    'ACT' : ['Manuka Oval'],
    'TAS' : ['York Park', 'Bellerive Oval'],
    'NT' : ['Marrara Oval', 'Traeger Park'],
    'NZ' : ['Wellington'],
    'CH': ['Jiangwan Stadium']
}

teams_loc = {
    'VIC' : ['RICH', 'COLL', 'MELB', 'ESS', 'NMFC', 
             'CARL', 'GEEL', 'STK', 'HAW', 'WB'],
    'SA' : ['ADEL', 'PORT'],
    'NSW' : ['GWS', 'SYD'],
    'QLD' : ['BL', 'GCFC'],
    'WA' : ['WCE', 'FRE'],
}

team_stadiums = {'RICH' : 'M.C.G.',
                 'COLL' : 'M.C.G.',
                 'MELB' : 'M.C.G.',
                 'ADEL' : 'Adelaide Oval',
                 'ESS' : 'Docklands',
                 'BL' : 'Gabba',
                 'NMFC' : 'Docklands',
                 'GWS' : 'Sydney Showground',
                 'WCE' : 'Perth Stadium',
                 'CARL' : 'Docklands',
                 'GEEL' : 'Kardinia Park',
                 'SYD' : 'S.C.G.',
                 'PORT' : 'Adelaide Oval',
                 'STK' : 'Docklands',
                 'GCFC' : 'Carrara',
                 'HAW' : 'M.C.G.',
                 'WB' : 'Docklands',
                 'FRE' : 'Perth Stadium'}

In [37]:
def interstate_det(venues_loc,teams_loc,data):
    
    '''
    Calculates whether or not the team is playing interstate.
    
    Parameters:
        - venues_loc: dictionary with venue location.
        - teams_loc: dictionary with team State.
        - data: data on which the function is run.
        
    Returns:
        - 1 if the location of the venue and team match, 0 otherwise.
        
    '''
    venue_key = None
    
    for key, values in venues_loc.items():
        if data['VENUE'] in values:
            venue_key = key
            break

    team_key = None
    for key, values in teams_loc.items():
        if data['TEAM'] in values:
            team_key = key
            break

    if venue_key == team_key:
        
        return 0
    else:
        
        return 1

def calculate_homeground(row):
    
    '''
    Calculates whether or not the team is playing in their homeground (1) or not (0)
    
    '''
    
    if row['VENUE'] == team_stadiums[row['TEAM']]:
        return 1
    elif row['SEASON'] < 2013 and row['TEAM'] == 'ADEL' and row['VENUE'] == 'Football Park':
        return 1
    elif row['SEASON'] < 2013 and row['TEAM'] == 'PORT' and row['VENUE'] == 'Football Park':
        return 1
    elif row['SEASON'] < 2017 and row['TEAM'] == 'FRE' and row['VENUE'] == 'Subiaco':
        return 1
    elif row['SEASON'] < 2017 and row['TEAM'] == 'WCE' and row['VENUE'] == 'Subiaco':
        return 1
    elif row['SEASON'] < 2013 and row['TEAM'] == 'GWS' and row['VENUE'] == 'Stadium Australia':
        return 1
    elif row['SEASON'] < 2015 and row['TEAM'] == 'SYD' and row['VENUE'] == 'Stadium Australia':
        return 1
    elif row['SEASON'] < 2013 and row['TEAM'] == 'GWS' and row['VENUE'] == 'Blacktown':
        return 1
    elif row['TEAM'] == 'WB' and row['VENUE'] == 'Eureka Stadium':
        return 1
    else:
        return 0
    

cols4 = ['HOME_CLANGERS_L4_CSUM_DIFF', 'AWAY_CLANGERS_L4_CSUM_DIFF',
        'HOME_FREES_AGAINST_L4_CSUM_DIFF', 'AWAY_FREES_AGAINST_L4_CSUM_DIFF',
        'HOME_TURNOVERS_L4_CSUM_DIFF', 'AWAY_TURNOVERS_L4_CSUM_DIFF', 
         'HOME_CONTEST_DEFENSIVE_LOSS_L4_CSUM_DIFF', 'AWAY_CONTEST_DEFENSIVE_LOSS_L4_CSUM_DIFF',
        'HOME_L4G_WINS', 'AWAY_L4G_WINS', 'HOME_DISPOSALS_L4_CSUM_DIFF', 'AWAY_DISPOSALS_L4_CSUM_DIFF',
        'HOME_DISPOSALS_EFFECTIVE_L4_CSUM_DIFF', 'AWAY_DISPOSALS_EFFECTIVE_L4_CSUM_DIFF',
        'HOME_KICKS_L4_CSUM_DIFF', 'AWAY_KICKS_L4_CSUM_DIFF', 'HOME_KICKS_EFFECTIVE_L4_CSUM_DIFF',
        'AWAY_KICKS_EFFECTIVE_L4_CSUM_DIFF', 'HOME_HANDBALLS_L4_CSUM_DIFF', 'AWAY_HANDBALLS_L4_CSUM_DIFF',
        'HOME_MARKS_L4_CSUM_DIFF','AWAY_MARKS_L4_CSUM_DIFF','HOME_MARKS_INSIDE50_L4_CSUM_DIFF',
         'AWAY_MARKS_INSIDE50_L4_CSUM_DIFF', 'HOME_MARKS_CONTESTED_L4_CSUM_DIFF',
         'AWAY_MARKS_CONTESTED_L4_CSUM_DIFF','HOME_MARKS_ONLEAD_L4_CSUM_DIFF',
         'AWAY_MARKS_ONLEAD_L4_CSUM_DIFF','HOME_MARKS_INTERCEPT_L4_CSUM_DIFF',
         'AWAY_MARKS_INTERCEPT_L4_CSUM_DIFF','HOME_BOUNCES_L4_CSUM_DIFF','AWAY_BOUNCES_L4_CSUM_DIFF',
        'HOME_METRES_GAINED_L4_CSUM_DIFF','AWAY_METRES_GAINED_L4_CSUM_DIFF','HOME_POSESSIONS_L4_CSUM_DIFF',
         'AWAY_POSESSIONS_L4_CSUM_DIFF','HOME_POSESSIONS_CONTESTED_L4_CSUM_DIFF',
         'AWAY_POSESSIONS_CONTESTED_L4_CSUM_DIFF','HOME_POSESSIONS_UNCONTESTED_L4_CSUM_DIFF',
         'AWAY_POSESSIONS_UNCONTESTED_L4_CSUM_DIFF','HOME_GETS_GROUNDBALL_L4_CSUM_DIFF',
         'AWAY_GETS_GROUNDBALL_L4_CSUM_DIFF','HOME_GETS_GROUNDBALL50_L4_CSUM_DIFF',
         'AWAY_GETS_GROUNDBALL50_L4_CSUM_DIFF','HOME_INSIDE50_L4_CSUM_DIFF','AWAY_INSIDE50_L4_CSUM_DIFF',
         'HOME_GOALS_SHOTS_L4_CSUM_DIFF','AWAY_GOALS_SHOTS_L4_CSUM_DIFF','HOME_SCORE_LAUNCHES_L4_CSUM_DIFF',
         'AWAY_SCORE_LAUNCHES_L4_CSUM_DIFF','HOME_CLEARANCES_L4_CSUM_DIFF','AWAY_CLEARANCES_L4_CSUM_DIFF',
        'HOME_CLEARANCES_CENTRE_L4_CSUM_DIFF','AWAY_CLEARANCES_CENTRE_L4_CSUM_DIFF',
        'HOME_CLEARANCES_STOPPAGE_L4_CSUM_DIFF','AWAY_CLEARANCES_STOPPAGE_L4_CSUM_DIFF',
        'HOME_HITOUTS_ADVANTAGE_L4_CSUM_DIFF','AWAY_HITOUTS_ADVANTAGE_L4_CSUM_DIFF',
        'HOME_HITOUTS_WIN_RATE_L4_CSUM_DIFF','AWAY_HITOUTS_WIN_RATE_L4_CSUM_DIFF',
        'HOME_CONTEST_OFFENSIVE_WIN_L4_CSUM_DIFF','AWAY_CONTEST_OFFENSIVE_WIN_L4_CSUM_DIFF',
        'HOME_TACKLES_L4_CSUM_DIFF','AWAY_TACKLES_L4_CSUM_DIFF','HOME_TACKLES_INSIDE50_L4_CSUM_DIFF',
         'AWAY_TACKLES_INSIDE50_L4_CSUM_DIFF','HOME_ONE_PERCENTERS_L4_CSUM_DIFF',
         'AWAY_ONE_PERCENTERS_L4_CSUM_DIFF','HOME_REBOUND_INSIDE50S_L4_CSUM_DIFF',
         'AWAY_REBOUND_INSIDE50S_L4_CSUM_DIFF','HOME_INTERCEPTS_L4_CSUM_DIFF','AWAY_INTERCEPTS_L4_CSUM_DIFF',
        'HOME_PRESSURE_L4_CSUM_DIFF','AWAY_PRESSURE_L4_CSUM_DIFF','HOME_PRESSURE_DEFENSEHALF_L4_CSUM_DIFF',
         'AWAY_PRESSURE_DEFENSEHALF_L4_CSUM_DIFF','HOME_SPOILS_L4_CSUM_DIFF','AWAY_SPOILS_L4_CSUM_DIFF',
        'HOME_DISPOSALS_EFFICIENCY_L4_CSUM_DIFF','AWAY_DISPOSALS_EFFICIENCY_L4_CSUM_DIFF',
        'HOME_KICKS_EFFICIENCY_L4_CSUM_DIFF','AWAY_KICKS_EFFICIENCY_L4_CSUM_DIFF',
        'HOME_KICK2HANDBALL_L4_CSUM_DIFF','AWAY_KICK2HANDBALL_L4_CSUM_DIFF',
        'HOME_POSESSIONS_CONTESTED_RATE_L4_CSUM_DIFF','AWAY_POSESSIONS_CONTESTED_RATE_L4_CSUM_DIFF',
        'HOME_GOALS_ACCURACY_L4_CSUM_DIFF','AWAY_GOALS_ACCURACY_L4_CSUM_DIFF',
        'HOME_HITOUTS_ADVANTAGE_RATE_L4_CSUM_DIFF','AWAY_HITOUTS_ADVANTAGE_RATE_L4_CSUM_DIFF',
        'HOME_CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM_DIFF','AWAY_CONTEST_OFFENSIVE_WIN_RATE_L4_CSUM_DIFF',
        'HOME_CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM_DIFF','AWAY_CONTEST_DEFENSIVE_LOSS_RATE_L4_CSUM_DIFF'] 



colscomp = ['HOME_CLANGERS_CSUM_DIFF', 'AWAY_CLANGERS_CSUM_DIFF', 'HOME_FREES_AGAINST_CSUM_DIFF',
            'AWAY_FREES_AGAINST_CSUM_DIFF', 'HOME_TURNOVERS_CSUM_DIFF', 'AWAY_TURNOVERS_CSUM_DIFF',
            'HOME_CONTEST_DEFENSIVE_LOSS_CSUM_DIFF', 'AWAY_CONTEST_DEFENSIVE_LOSS_CSUM_DIFF',
            'HOME_WINS_CUMMULATIVE_DIFF', 'AWAY_WINS_CUMMULATIVE_DIFF',
            'HOME_DISPOSALS_CSUM_DIFF', 'AWAY_DISPOSALS_CSUM_DIFF',
            'HOME_DISPOSALS_EFFECTIVE_CSUM_DIFF', 'AWAY_DISPOSALS_EFFECTIVE_CSUM_DIFF',
            'HOME_KICKS_CSUM_DIFF', 'AWAY_KICKS_CSUM_DIFF', 'HOME_KICKS_EFFECTIVE_CSUM_DIFF',
            'AWAY_KICKS_EFFECTIVE_CSUM_DIFF', 'HOME_HANDBALLS_CSUM_DIFF', 'AWAY_HANDBALLS_CSUM_DIFF',
           'HOME_MARKS_CSUM_DIFF', 'AWAY_MARKS_CSUM_DIFF', 'HOME_MARKS_INSIDE50_CSUM_DIFF',
            'AWAY_MARKS_INSIDE50_CSUM_DIFF','HOME_MARKS_CONTESTED_CSUM_DIFF','AWAY_MARKS_CONTESTED_CSUM_DIFF',
            'HOME_MARKS_ONLEAD_CSUM_DIFF','AWAY_MARKS_ONLEAD_CSUM_DIFF','HOME_MARKS_INTERCEPT_CSUM_DIFF',
            'AWAY_MARKS_INTERCEPT_CSUM_DIFF', 'HOME_BOUNCES_CSUM_DIFF','AWAY_BOUNCES_CSUM_DIFF',
            'HOME_METRES_GAINED_CSUM_DIFF','AWAY_METRES_GAINED_CSUM_DIFF','HOME_POSESSIONS_CSUM_DIFF',
            'AWAY_POSESSIONS_CSUM_DIFF','HOME_POSESSIONS_CONTESTED_CSUM_DIFF',
            'AWAY_POSESSIONS_CONTESTED_CSUM_DIFF', 'HOME_POSESSIONS_UNCONTESTED_CSUM_DIFF',
            'AWAY_POSESSIONS_UNCONTESTED_CSUM_DIFF','HOME_GETS_GROUNDBALL_CSUM_DIFF',
            'AWAY_GETS_GROUNDBALL_CSUM_DIFF', 'HOME_GETS_GROUNDBALL50_CSUM_DIFF',
            'AWAY_GETS_GROUNDBALL50_CSUM_DIFF','HOME_INSIDE50_CSUM_DIFF','AWAY_INSIDE50_CSUM_DIFF',
            'HOME_GOALS_SHOTS_CSUM_DIFF','AWAY_GOALS_SHOTS_CSUM_DIFF','HOME_SCORE_LAUNCHES_CSUM_DIFF',
            'AWAY_SCORE_LAUNCHES_CSUM_DIFF','HOME_CLEARANCES_CSUM_DIFF','AWAY_CLEARANCES_CSUM_DIFF',
            'HOME_CLEARANCES_CENTRE_CSUM_DIFF','AWAY_CLEARANCES_CENTRE_CSUM_DIFF',
            'HOME_CLEARANCES_STOPPAGE_CSUM_DIFF','AWAY_CLEARANCES_STOPPAGE_CSUM_DIFF',
            'HOME_HITOUTS_ADVANTAGE_CSUM_DIFF','AWAY_HITOUTS_ADVANTAGE_CSUM_DIFF',
            'HOME_HITOUTS_WIN_RATE_CSUM_DIFF','AWAY_HITOUTS_WIN_RATE_CSUM_DIFF',
            'HOME_CONTEST_OFFENSIVE_WIN_CSUM_DIFF','AWAY_CONTEST_OFFENSIVE_WIN_CSUM_DIFF',
            'HOME_TACKLES_CSUM_DIFF','AWAY_TACKLES_CSUM_DIFF','HOME_TACKLES_INSIDE50_CSUM_DIFF',
            'AWAY_TACKLES_INSIDE50_CSUM_DIFF','HOME_ONE_PERCENTERS_CSUM_DIFF','AWAY_ONE_PERCENTERS_CSUM_DIFF',
            'HOME_REBOUND_INSIDE50S_CSUM_DIFF','AWAY_REBOUND_INSIDE50S_CSUM_DIFF',
            'HOME_INTERCEPTS_CSUM_DIFF','AWAY_INTERCEPTS_CSUM_DIFF','HOME_PRESSURE_CSUM_DIFF',
            'AWAY_PRESSURE_CSUM_DIFF','HOME_PRESSURE_DEFENSEHALF_CSUM_DIFF',
            'AWAY_PRESSURE_DEFENSEHALF_CSUM_DIFF','HOME_SPOILS_CSUM_DIFF','AWAY_SPOILS_CSUM_DIFF',
            'HOME_DISPOSALS_EFFICIENCY_CSUM_DIFF','AWAY_DISPOSALS_EFFICIENCY_CSUM_DIFF',
            'HOME_KICKS_EFFICIENCY_CSUM_DIFF','AWAY_KICKS_EFFICIENCY_CSUM_DIFF',
            'HOME_KICK2HANDBALL_CSUM_DIFF','AWAY_KICK2HANDBALL_CSUM_DIFF',
            'HOME_POSESSIONS_CONTESTED_RATE_CSUM_DIFF','AWAY_POSESSIONS_CONTESTED_RATE_CSUM_DIFF',
            'HOME_GOALS_ACCURACY_CSUM_DIFF','AWAY_GOALS_ACCURACY_CSUM_DIFF',
            'HOME_HITOUTS_ADVANTAGE_RATE_CSUM_DIFF','AWAY_HITOUTS_ADVANTAGE_RATE_CSUM_DIFF',
            'HOME_CONTEST_OFFENSIVE_WIN_RATE_CSUM_DIFF','AWAY_CONTEST_OFFENSIVE_WIN_RATE_CSUM_DIFF',
            'HOME_CONTEST_DEFENSIVE_LOSS_RATE_CSUM_DIFF','AWAY_CONTEST_DEFENSIVE_LOSS_RATE_CSUM_DIFF'
           ]
    
def data_preparation_BT_homeaway(data, season_range = None, round_range = None , cuatro_comp = None):
    
    '''
    Prepares data for modelling Team-Specific, Time-Variant BT expansions.
    
    Parameters:
        - data: pre-processed data.
        - season_range: seasons to filter the data.
        - round_range: rounds to filter the data.
        - cuatro_comp (str): string to decide to keep cumulatives over the season or 
        the last 4 games.
        
    Returns:
        - homies: data for home teams.
        - awayers: data for away teams.
        
    '''
    
    if season_range is not None:
        data = data[data['SEASON'].isin(season_range)]
    
    if round_range is not None:
        data = data[data['ROUND_NUMBER'].isin(round_range)]
    
    data = data[data['DRAW'] == 0]
    
    if cuatro_comp == 'L4':
        data = data.drop(columns=colscomp)
    elif cuatro_comp == 'ALL':
        data = data.drop(columns=cols4)
    
    home_cols = [col for col in data.columns if ('HOME_' in col) or (col in ['SEASON', 'ROUND_NUMBER', 'VENUE'])]
    away_cols = [col for col in data.columns if ('AWAY_' in col) or (col in ['SEASON', 'ROUND_NUMBER', 'VENUE'])]
    
    homies = data[home_cols].copy(deep = True)
    awayers = data[away_cols].copy(deep = True)
    
    homies['AT_HOME'] = 1
    awayers['AT_HOME'] = 0

    homies.columns = homies.columns.str.replace('HOME_', '')
    awayers.columns = awayers.columns.str.replace('AWAY_', '')
    
    homies['HOMEGROUND'] = homies.apply(calculate_homeground, axis=1)
    awayers['HOMEGROUND'] = awayers.apply(calculate_homeground, axis=1)
    homies['INTERSTATE'] = homies.apply(lambda x: interstate_det(venues_loc, teams_loc, x), axis=1)
    awayers['INTERSTATE'] = awayers.apply(lambda x: interstate_det(venues_loc, teams_loc, x), axis=1)

    homies.sort_index(inplace = True)
    awayers.sort_index(inplace = True)

    return homies, awayers

### Run Processing

In [43]:
home, away = data_preparation_BT_homeaway(resultss_before_modelling, season_range = [2015], cuatro_comp = 'L4')

In [ ]:
home

In [ ]:
away

In [ ]:
away[['LADDERLY_POSITION_DIFF']]

In [45]:
for c in away.columns:
    print(c)

SEASON
ROUND_NUMBER
VENUE
TEAM
POINTS
WIN
CONSECUTIVE_WINS
CONSECUTIVE_LOSSES
LG_WON
L4G_WINS
LADDER_POSITION_DIFF
LADDERLY_POSITION_DIFF
POINTSAGAINST_DIFF
CLANGERS_L4_CSUM_DIFF
FREES_AGAINST_L4_CSUM_DIFF
TURNOVERS_L4_CSUM_DIFF
CONTEST_DEFENSIVE_LOSS_L4_CSUM_DIFF
PERCENTAGE_DIFF
POINTSFOR_DIFF
DISPOSALS_L4_CSUM_DIFF
DISPOSALS_EFFECTIVE_L4_CSUM_DIFF
KICKS_L4_CSUM_DIFF
KICKS_EFFECTIVE_L4_CSUM_DIFF
HANDBALLS_L4_CSUM_DIFF
MARKS_L4_CSUM_DIFF
MARKS_INSIDE50_L4_CSUM_DIFF
MARKS_CONTESTED_L4_CSUM_DIFF
MARKS_ONLEAD_L4_CSUM_DIFF
MARKS_INTERCEPT_L4_CSUM_DIFF
BOUNCES_L4_CSUM_DIFF
METRES_GAINED_L4_CSUM_DIFF
POSESSIONS_L4_CSUM_DIFF
POSESSIONS_CONTESTED_L4_CSUM_DIFF
POSESSIONS_UNCONTESTED_L4_CSUM_DIFF
GETS_GROUNDBALL_L4_CSUM_DIFF
GETS_GROUNDBALL50_L4_CSUM_DIFF
INSIDE50_L4_CSUM_DIFF
GOALS_SHOTS_L4_CSUM_DIFF
SCORE_LAUNCHES_L4_CSUM_DIFF
CLEARANCES_L4_CSUM_DIFF
CLEARANCES_CENTRE_L4_CSUM_DIFF
CLEARANCES_STOPPAGE_L4_CSUM_DIFF
HITOUTS_ADVANTAGE_L4_CSUM_DIFF
HITOUTS_WIN_RATE_L4_CSUM_DIFF
CONTEST_OFFENSIVE_WI

# Prepare and Save CSVs

In [38]:
def prep_BT_TS_TV(data, cuatro_comp):
    
    '''
    Runs data data_preparation_BT_homeaway to return a list of datasets ready to
    model.
        
    '''
    
    home_list = []
    away_list = []
    
    home_full, away_full = data_preparation_BT_homeaway(data, cuatro_comp = cuatro_comp)
    tempos = data['SEASON'].unique()
    
    for season in tempos:
        print(season)
        home_season, away_season = data_preparation_BT_homeaway(data, 
                                                                cuatro_comp = cuatro_comp, 
                                                                season_range = [season])
        # print(season_df.shape)
        # season_df = season_df.drop(columns = uncols)
        home_list.append(home_season)
        away_list.append(away_season)
    
    home_list.append(home_full)
    away_list.append(away_full)
    print(len(home_list), len(away_list))
    
    return home_list, away_list

def save2excel_TS_TV(dataset_list, casa_fuera,cuatro_comp):
    for i, data in enumerate(dataset_list):
        season = 2015 + i
        if season == 2024:
            season = 'FULL'
        filename = f'{casa_fuera}_{cuatro_comp}_AFL_TS_TV_{season}.xlsx'
        data.to_excel(filename, index=False)
        print(f"Saved data for season {season} to {filename}")

In [39]:
homed, awayd = prep_BT_TS_TV(resultss_before_modelling, 'ALL')

2015
2016
2017
2018
2019
2020
2021
2022
2023
10 10


In [46]:
for c in awayd[0].columns:
    print(c)

SEASON
ROUND_NUMBER
VENUE
TEAM
POINTS
WIN
CONSECUTIVE_WINS
CONSECUTIVE_LOSSES
LG_WON
LADDER_POSITION_DIFF
LADDERLY_POSITION_DIFF
POINTSAGAINST_DIFF
CLANGERS_CSUM_DIFF
FREES_AGAINST_CSUM_DIFF
TURNOVERS_CSUM_DIFF
CONTEST_DEFENSIVE_LOSS_CSUM_DIFF
WINS_CUMMULATIVE_DIFF
PERCENTAGE_DIFF
POINTSFOR_DIFF
DISPOSALS_CSUM_DIFF
DISPOSALS_EFFECTIVE_CSUM_DIFF
KICKS_CSUM_DIFF
KICKS_EFFECTIVE_CSUM_DIFF
HANDBALLS_CSUM_DIFF
MARKS_CSUM_DIFF
MARKS_INSIDE50_CSUM_DIFF
MARKS_CONTESTED_CSUM_DIFF
MARKS_ONLEAD_CSUM_DIFF
MARKS_INTERCEPT_CSUM_DIFF
BOUNCES_CSUM_DIFF
METRES_GAINED_CSUM_DIFF
POSESSIONS_CSUM_DIFF
POSESSIONS_CONTESTED_CSUM_DIFF
POSESSIONS_UNCONTESTED_CSUM_DIFF
GETS_GROUNDBALL_CSUM_DIFF
GETS_GROUNDBALL50_CSUM_DIFF
INSIDE50_CSUM_DIFF
GOALS_SHOTS_CSUM_DIFF
SCORE_LAUNCHES_CSUM_DIFF
CLEARANCES_CSUM_DIFF
CLEARANCES_CENTRE_CSUM_DIFF
CLEARANCES_STOPPAGE_CSUM_DIFF
HITOUTS_ADVANTAGE_CSUM_DIFF
HITOUTS_WIN_RATE_CSUM_DIFF
CONTEST_OFFENSIVE_WIN_CSUM_DIFF
TACKLES_CSUM_DIFF
TACKLES_INSIDE50_CSUM_DIFF
ONE_PERCENTERS_CS

In [42]:
save2excel_TS_TV(homed, 'HOME', 'ALL')

Saved data for season 2015 to HOME_ALL_AFL_TS_TV_2015.xlsx
Saved data for season 2016 to HOME_ALL_AFL_TS_TV_2016.xlsx
Saved data for season 2017 to HOME_ALL_AFL_TS_TV_2017.xlsx
Saved data for season 2018 to HOME_ALL_AFL_TS_TV_2018.xlsx
Saved data for season 2019 to HOME_ALL_AFL_TS_TV_2019.xlsx
Saved data for season 2020 to HOME_ALL_AFL_TS_TV_2020.xlsx
Saved data for season 2021 to HOME_ALL_AFL_TS_TV_2021.xlsx
Saved data for season 2022 to HOME_ALL_AFL_TS_TV_2022.xlsx
Saved data for season 2023 to HOME_ALL_AFL_TS_TV_2023.xlsx
Saved data for season FULL to HOME_ALL_AFL_TS_TV_FULL.xlsx


In [43]:
save2excel_TS_TV(awayd, 'AWAY', 'ALL')

Saved data for season 2015 to AWAY_ALL_AFL_TS_TV_2015.xlsx
Saved data for season 2016 to AWAY_ALL_AFL_TS_TV_2016.xlsx
Saved data for season 2017 to AWAY_ALL_AFL_TS_TV_2017.xlsx
Saved data for season 2018 to AWAY_ALL_AFL_TS_TV_2018.xlsx
Saved data for season 2019 to AWAY_ALL_AFL_TS_TV_2019.xlsx
Saved data for season 2020 to AWAY_ALL_AFL_TS_TV_2020.xlsx
Saved data for season 2021 to AWAY_ALL_AFL_TS_TV_2021.xlsx
Saved data for season 2022 to AWAY_ALL_AFL_TS_TV_2022.xlsx
Saved data for season 2023 to AWAY_ALL_AFL_TS_TV_2023.xlsx
Saved data for season FULL to AWAY_ALL_AFL_TS_TV_FULL.xlsx


In [ ]:
homed[0].columns

In [ ]:
awayd[0]